<a href="https://colab.research.google.com/github/yoloso/hello-world/blob/master/Week8_Homework_Pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 8 Homework: Pneumonia Classification on X-Ray Images

An application of Computer Vision for healthcare is training models on X-ray images to detect Pneumonia. In this homework we will use CNNs and transfer learning to detect pnemonia in Kaggle's Chest X-Ray Images dataset.

For this homework to run, you must download the data zip file yourself as such:
- Go to [this Kaggle link](https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia) and download the dataset (you may need to create a Kaggle account)
- Rename the zip file to "chest_xray.zip" and upload it to your Google Drive at the root Folder

Before you run the below cell, be sure to make sure that you are using GPU (Edit->Notebook settings) because we will be training larger networks than before.

In [ ]:
import requests, zipfile, io
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
from io import BytesIO

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalAveragePooling2D, Dense
from keras import optimizers
from keras.models import Model

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/chest_xray.zip

## Part 1: Visualize the Data

Just as in previous assignments, we begin by looking to better understand our data. Here, your job is to use the function *visualize_train_image* below to display a finite number of images. You can find image file names in `filtered_train_data` above.

In [ ]:
def visualize_train_image(set_name):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(12, 6)

    pneumonia_images = os.path.join('chest_xray', set_name, 'PNEUMONIA')
    pneumonia_path = os.path.join(pneumonia_images, os.listdir(pneumonia_images)[0])
      
    im = Image.open(pneumonia_path)
    ax1.imshow(im)

    normal_images = os.path.join('chest_xray', set_name, 'NORMAL')
    normal_path = os.path.join(normal_images, os.listdir(normal_images)[0])
      
    im = Image.open(normal_path)
    ax2.imshow(im)

    plt.show()

In [ ]:
### BEGIN YOUR CODE ###
### END YOUR CODE ###

## Part 2: Exploring CNN Architectures 

At this point in the course, you've had the opportunity to construct and train your own convolutional neural networks with extra bells and whistles like pooling or batch normalization layers. Your CNNs were likely only a few layers deep, which constrains the representative power of the models. 

Thankfully, researchers around the world have been working on stacking more and more layers to create deeper architectures. However, early on, these researchers found that deeper networks actually performed worse. Why could this be?

The authors of an architecture called [ResNet](https://arxiv.org/abs/1512.03385) observed something simple: direct mappings are hard to learn. Instead of trying to learn an underlying mapping from $x$ to $f(x)$, we can learn the difference between the two, or the “residual.” Then, to calculate $f(x)$, we can just add the residual to the input. Say the residual is $r(x) = f(x) - x$. Now, instead of trying to learn $f(x)$ directly, our networks are trying to learn $r(x)+x$. You can check out this [blog post](https://towardsdatascience.com/an-intuitive-guide-to-deep-network-architectures-65fdc477db41) to learn more.

We'll be exploring a 50-layer ResNet CNN architecture in Keras in Part 3.

Before we jump into training our ResNet though, we need to load our data in with Keras data generators. Data generators are cool because they quickly iterate through your sets in batches. The batch size is a hyperparameter you can tune. We've put together that code for you. You can go ahead and play around with the batch size later when submitting your final model (note that batch sizes above 64 might cause memory errors, because very large batches will not fit on GPU).

In [ ]:
### BEGIN YOUR CODE ###
batch_size = 32 
### END YOUR CODE ###

def get_data_generator(set_name, batch_size):
    datagen = ImageDataGenerator(rescale = 1./255, horizontal_flip = set_name == 'train')
    generator = datagen.flow_from_directory('./chest_xray/' + set_name,
                                            target_size = (224, 224),
                                            batch_size = batch_size,
                                            class_mode = 'categorical')
    return generator
    
train_generator = get_data_generator('train', batch_size)
val_generator = get_data_generator('val', batch_size)
test_generator = get_data_generator('test', batch_size)

num_classes = 2

FileNotFoundError: ignored

We can see that there are 2 classes.

Keras has an easy API for loading the ResNet-50. You can play around with the learning rate.

In [ ]:
### BEGIN YOUR CODE ###
lr = 1e-3
### END YOUR CODE ###

def get_base_resnet_model(lr):
    model = keras.applications.ResNet50(include_top = True, weights = None, 
                                        classes = num_classes, input_shape=(224, 224, 3), 
                                        input_tensor = None)

    optim = optimizers.Adam(lr = lr)
    model.compile(optimizer = optim, loss = 'categorical_crossentropy', metrics=['accuracy'])

    return model
    
base_resnet_model = get_base_resnet_model(lr)
base_resnet_model.summary()

Now that we have our 50-layer architecture set up, let's train it for 5 epochs! Refer to the [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit_generator) to fit the model to the data from your generators. This will take around 10 minutes (make sure to run on GPU)! The training script will print out loss and accuracies as it trains–note that accuracies will be lower than you've seen before because we have many more output classes and we are only training for one epoch.

In [ ]:
### BEGIN YOUR CODE ###

### END YOUR CODE ###

## Part 3: Transfer Learning

Good job training your first super-deep network! For some historical context, ResNet was created for the ImageNet Large Scale Visual Recognition Challenge (ILSVRC). From the ImageNet [website](http://image-net.org/challenges/LSVRC/), "ILSVRC evaluates algorithms for object detection and image classification at large scale. One high level motivation is to allow researchers to compare progress... across a wider variety of objects -- taking advantage of the quite expensive labeling effort." In other words, ImageNet is a huge labeled dataset that researchers can use to benchmark their architectures. 

After ResNet-50 is trained on ImageNet, we can save the weights and use them to bootstrap another task with the same architecture. This technique is broadly called **transfer learning**. As discussed in this [blog post](https://machinelearningmastery.com/transfer-learning-for-deep-learning/), "In transfer learning, we first train a base network on a base dataset and task, and then we repurpose the learned features, or transfer them, to a second target network to be trained on a target dataset and task. This process will tend to work if the features are general, meaning suitable to both base and target tasks, instead of specific to the base task." In this case, the task is image classification on a different dataset.

In Part 4, you'll have the opportunity to use a ResNet initialized with **pretrained weights** and see how it does on your task. Note that the ImageNet dataset has 1000 classes but our dataset only has 31 classes, so we'll have to replace the last layer of our ResNet to reflect that difference. Again, you can play around with the learning rate.



In [ ]:
### BEGIN YOUR CODE ###
lr = 1e-3
### END YOUR CODE ###

def get_pretrained_resnet_model(lr):
    # here, we pass in weights = 'imagenet' instead of weights = None
    pretrained_resnet = keras.applications.ResNet50(include_top = False, weights = 'imagenet', 
                                                    classes = num_classes, input_shape = (224, 224, 3), 
                                                    input_tensor = None)
    # replace last layer (including the pooling)
    h = GlobalAveragePooling2D()(pretrained_resnet.output)
    y_hat = Dense(num_classes, activation = 'softmax', name = 'fc1000')(h)
    
    model = Model(inputs = pretrained_resnet.input, outputs = y_hat)

    optim = optimizers.Adam(lr = lr)
    model.compile(optimizer = optim, loss = 'categorical_crossentropy', metrics=['accuracy'])

    return model
    
pretrained_resnet_model = get_pretrained_resnet_model(lr)
pretrained_resnet_model.summary()

Train your new pretrained model for 1 epoch. Refer to the code for *base_resnet_model* above to see how to use *fit_generator*!

In [ ]:
### BEGIN YOUR CODE ###

### END YOUR CODE ###

## Part 4: Transfer Learning with Frozen Layers

As you might have noticed, training these 50-layer ResNets end-to-end takes quite a bit more time than our previous smaller models. One of the big benefits of transfer learning is that it can oftentimes speed up training. 

In CNNs, features are more generic in early layers and more original-dataset-specific in later layers. This means that the initial layers of the ResNet model are still useful for this new task. So, we can choose to keep our ImageNet weights in earlier layers and only finetune them in later layers. We do this by freezing earlier layers and making them untrainable. We thus introduce an additional hyperparameter: the number of frozen layers. Note that due to different definitions for what a "layer" means, model.layers contains 177 layers.

In [ ]:
### BEGIN YOUR CODE ###
lr = 1e-3
num_freeze_layers = 30
### END YOUR CODE ###

def get_pretrained_frozen_resnet_model(lr, num_freeze_layers):
    pretrained_resnet = keras.applications.ResNet50(include_top = False, weights = 'imagenet', 
                                                    classes = 31, input_shape = (224, 224, 3), 
                                                    input_tensor = None)
    # replace last layer (including the pooling)
    h = GlobalAveragePooling2D()(pretrained_resnet.output)
    y_hat = Dense(num_classes, activation = 'softmax', name = 'fc1000')(h)
    
    model = Model(inputs = pretrained_resnet.input, outputs = y_hat)

    optim = optimizers.Adam(lr = lr)
    model.compile(optimizer = optim, loss = 'categorical_crossentropy', metrics=['accuracy'])
    
    print('Freezing %d of %d model layers...' % (num_freeze_layers, len(model.layers)))
    
    if not num_freeze_layers:
        for layer in model.layers[:num_freeze_layers]:
            layer.trainable = False
        for layer in model.layers[num_freeze_layers:]:
            layer.trainable = True

    return model
    
pretrained_frozen_resnet_model = get_pretrained_frozen_resnet_model(lr, num_freeze_layers)
pretrained_frozen_resnet_model.summary()

Train your new pretrained model with frozen layers for 5 epoch. Notice that training is a bit faster now, depending on how many layers you are freezing, since we are training fewer layers.

In [ ]:
### BEGIN YOUR CODE ###

### END YOUR CODE ###

## Part 5: Evaluation

At this point, we've trained a base 50-layer ResNet and a pretrained ResNet initialized with ImageNets weights. We've also finetuned a pretrained ResNet in Part 5. After completing hyperparameter tuning above (and also training for more epochs, if you like), pick the best of your three models according to validation performance and run evaluation on your test set. Note that you do not have to wait for training to complete to get an idea of what hyperparameters are doing well–you can keep track of how quickly the loss decreases.

Remember: to avoid bias, only run evaluation on the test set once! Use validation accuracy (reported at the end of an epoch) to tune hyperparameters.




In [ ]:
### BEGIN YOUR CODE ###
# Save your best model to best_model here

loss, acc = best_model.evaluate_generator(test_generator, len(test_generator))

print('Your best test loss was', loss)
print('Your best test accuracy was', acc)
### END YOUR CODE ###

## And that's a wrap!

The skills covered in this notebook are very critical for deep learning in practice, because machine learning engineers don't want to keep reinventing the wheel. Good work finishing up this assignment! 